In [14]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch
import os
import pathlib

In [20]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
print("ok")


OSError: Could not find a suitable TLS CA certificate bundle, invalid path: /Users/holgermaxfloelyng/Desktop/BioMed/MSc_Biomed/SEM_3/project/mlops_project/.venv/lib/python3.12/site-packages/certifi/cacert.pem

In [ ]:
from pathlib import Path
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader

root = Path.cwd().parent.parent.resolve()
DATA_DIR = root / "mlops_project" / "data" / "Voice_of_Birds"

AUDIO_EXTS = {".wav", ".mp3", ".flac", ".ogg", ".m4a"}

print("Data dir:", DATA_DIR)

# 1) Index the dataset: (filepath, label_id)
class_dirs = sorted([p for p in DATA_DIR.iterdir() if p.is_dir()])
classes = [p.name for p in class_dirs]
class_to_id = {c: i for i, c in enumerate(classes)}

items = []
for cdir in class_dirs:
    for f in cdir.rglob("*"):
        if f.suffix.lower() in AUDIO_EXTS:
            items.append((str(f), class_to_id[cdir.name]))

print("Classes:", len(classes))
print("Files:", len(items))
print("Example:", items[0])

# 2) Dataset
class BirdAudioDataset(Dataset):
    def __init__(self, items, target_sr=16000):
        self.items = items
        self.target_sr = target_sr

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        path, y = self.items[idx]
        wav, sr = torchaudio.load(path)            # [C, T]
        if wav.shape[0] > 1:
            wav = wav.mean(dim=0, keepdim=True)    # mono
        if sr != self.target_sr:
            wav = torchaudio.functional.resample(wav, sr, self.target_sr)
        return wav.squeeze(0), y                   # ([T], int)

# 3) Collate: pad to same length in batch
def collate_pad(batch):
    xs, ys = zip(*batch)
    lengths = torch.tensor([x.shape[0] for x in xs], dtype=torch.long)
    xpad = torch.nn.utils.rnn.pad_sequence(xs, batch_first=True)  # [B, Tmax]
    y = torch.tensor(ys, dtype=torch.long)
    return {"input_values": xpad, "lengths": lengths, "labels": y}

ds = BirdAudioDataset(items, target_sr=16000)
dl = DataLoader(ds, batch_size=8, shuffle=True, collate_fn=collate_pad, num_workers=0)

batch = next(iter(dl))
print(batch["input_values"].shape)  # [B, Tmax]
print(batch["labels"][:5])
print(batch["lengths"][:5])

# Optional: map label ids back to names
id_to_class = {v: k for k, v in class_to_id.items()}
print("First label name:", id_to_class[int(batch["labels"][0])])


Data dir: /Users/holgermaxfloelyng/Desktop/BioMed/MSc_Biomed/SEM_3/project/mlops_project/data/Voice_of_Birds
Classes: 5
Files: 126
Example: ('/Users/holgermaxfloelyng/Desktop/BioMed/MSc_Biomed/SEM_3/project/mlops_project/data/Voice_of_Birds/Andean Guan_sound/Andean Guan23.mp3', 0)


RuntimeError: Numpy is not available